<a href="https://colab.research.google.com/github/Rohan641/Foreign_ExchangeRate_Forecast/blob/master/Forex_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the libraries
!pip install numpy
!pip install pandas
!pip install matplotlib

Data Procession 

The Data here is not present in a single sheet or a file. It's in the form of  a Workbook. Apart from the existing data, it requires other variable like SMA's, Returns, LastDay Closing Price etc. So here I've made use of Python Functions to create them and then implement it all together using main().

In [0]:
#Load all the reqd libs


import numpy as np

import pandas as pd

import matplotlib.pyplot as plt


    

In [6]:
from google.colab import files

uploaded = files.upload()

Saving data.xlsx to data.xlsx


In [0]:
# Reads an Excel file 

# return: data = ['date', 'last price']


def loadExcelData(fileName, sheetName):

  data = pd.DataFrame(pd.read_excel(fileName, sheetName), columns= ['Date', 'Last Price']) # creates a dataframe on reading excel files
	
  return data.values


cny_data = loadExcelData('data.xlsx','USDCNY')

jpy_data = loadExcelData('data.xlsx','USDJPY')

eur_data = loadExcelData('data.xlsx','USDEUR')

gbp_data = loadExcelData('data.xlsx','USDGBP')



In [0]:
#Return values(r2, r3, r4, ..., rn)


# construct rs
# r0 = None
# start from r1 = 100*log(p1-p0)


def construct_return(data):

  
  p = data[:,1].astype(float) # this line converts exchange price to float type
	
  r = np.zeros_like(p) # this creates an empty np obj of size p i.e 3521
	
  r[0] = None # set first elm to None
	
  r[1:] = 100.0 * (np.log(p[1:])-np.log(p[:-1])) # this helps in computing return
	
  return r


cny_r = construct_return(cny_data)

jpy_r = construct_return(jpy_data)

eur_r = construct_return(eur_data)

gbp_r = construct_return(gbp_data)



#p = cny_data[:,1].astype(float) # this line converts exchange price to float type

#r = np.zeros_like(p) # this creates an empty np obj of size p i.e 3521


In [0]:
# for example: SMA5

# in the form of (r2, (r2+r3)/2, (r2+r3+r4)/3, (r2+r3+r4+r5)/4, (r2+r3+r4+r5+r6)/5, 

# (r3+r4+r5+r6+r7)/5, ...)

def construct_SMA(r,degree):

  
  num_days = r.shape[0] # this actually gives the length of r
	
  sma = np.zeros_like(r) # now you create another empty obj of r shape
	
  for i in range(num_days):
	
    if i == 0:  # this is for the first element
	
        sma[i] = None
	
    elif i < degree:
	
        sma[i] = np.mean(r[1:i+1])# this is to calc moving avg 
	
    else:
	
        sma[i] = np.mean(r[i-degree+1:i+1]) # this after first iteration like if i > degree, use (i-degree+1)
	
  
  return sma



# construct sma

cny_sma5 = construct_SMA(cny_r,5)

jpy_sma5 = construct_SMA(jpy_r,5)

eur_sma5 = construct_SMA(eur_r,5)

gbp_sma5 = construct_SMA(gbp_r,5)



In [0]:
# return last day's closing price

def construct_lastDay(r):

  lDay = np.zeros_like(r)
	
  lDay[0:2] = None
	
  lDay[2:] = r[1:-1]
	

  return lDay



# construct last-day-closing-price structure
cny_lDay = construct_lastDay(cny_r)


jpy_lDay = construct_lastDay(jpy_r)


eur_lDay = construct_lastDay(eur_r)


gbp_lDay = construct_lastDay(gbp_r)




In [0]:
# construct X

# [x1 x2 x3 x4 x5 x6 x7]

# [r SMA5 SMA10 SMA20 SMA60 SMA120 lDay]  

def construct_X(data):

  x1 = construct_return(data).reshape((-1,1)) # shape (-1,1 ) means it creates 1 column and if its (1, -1) then 1 row
	
  x2 = construct_SMA(x1,5).reshape((-1,1)) # SMA of 5 Days
	
  x3 = construct_SMA(x1,10).reshape((-1,1))
	
  x4 = construct_SMA(x1,20).reshape((-1,1))
	
  x5 = construct_SMA(x1,60).reshape((-1,1))
	
  x6 = construct_SMA(x1,120).reshape((-1,1))
	
  x7 = construct_lastDay(x1).reshape((-1,1)) 
	
  x8 = np.log(data[:,1].astype(float)).reshape((-1,1)) # I didn't understand this code even x14 and x15
	
  x9 = construct_SMA(x8,5).reshape((-1,1))
	
  x10 = construct_SMA(x8, 10).reshape((-1, 1))
	
  x11 = construct_SMA(x8, 20).reshape((-1, 1))
	
  x12 = construct_SMA(x8, 60).reshape((-1, 1))
	
  x13 = construct_SMA(x8, 220).reshape((-1, 1))
	
  x14= np.log(np.square(data[:,1].astype(float)).reshape((-1,1)))
	
  x15 = np.log(data[:, 1].astype(float).reshape((-1, 1))**3)
	
  X = np.hstack((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15)) # This stacks up the entire thing horizontally
	
  X = X[2:-1,:] # 2nd row to last elem + all columns

  
  return(X)


In [0]:
# construct y

# y0 = None

# y1 = sign(r1)

def construct_y(data):

  r = construct_return(data)
	
  y = np.sign(r[3:]) # sign returns the sign of the array element 
  
  # starts with index == 3, bcoz 0th Index = None, 1st is nan and 2nd is starting/first elem
  
  # sign return -1 if r < 0, 1 if r > 0 and 0 when y == 0
  
  
  zero_index = np.where(y == 0)[0] # conditional numpy array
	
  y[zero_index] = 1
	
  return y

cny_y = construct_y(cny_data)

jpy_y = construct_y(jpy_data)	

eur_y = construct_y(eur_data)

gbp_y = construct_y(gbp_data)


In [16]:
# The main method


def main():

  # load data = ['date', 'last price']
	cny_data = loadExcelData('data.xlsx','USDCNY')
	jpy_data = loadExcelData('data.xlsx','USDJPY')
	eur_data = loadExcelData('data.xlsx','USDEUR')
	gbp_data = loadExcelData('data.xlsx','USDGBP')
	
	# Fill data in the empty dates (weekends) with the average
	# of porior and posterior data
	# cny_data = fillEmptyDates(cny_data)
	# jpy_data = fillEmptyDates(jpy_data)
	# eur_data = fillEmptyDates(eur_data)
  	# gbp_data = fillEmptyDates(gbp_data)

    
	# construct retrun values (r2,...,rn)
	cny_r = construct_return(cny_data)
	jpy_r = construct_return(jpy_data)
	eur_r = construct_return(eur_data)
	gbp_r = construct_return(gbp_data)

  
	# construct sma
	cny_sma5 = construct_SMA(cny_r,5)
	jpy_sma5 = construct_SMA(jpy_r,5)
	eur_sma5 = construct_SMA(eur_r,5)
	gbp_sma5 = construct_SMA(gbp_r,5)

	
  # construct last-day-closing-price structure
	cny_lDay = construct_lastDay(cny_r)
	jpy_lDay = construct_lastDay(jpy_r)
	eur_lDay = construct_lastDay(eur_r)
	gbp_lDay = construct_lastDay(gbp_r)

  
	# construct y 
	cny_y = construct_y(cny_data)
	jpy_y = construct_y(jpy_data)	
	eur_y = construct_y(eur_data)
	gbp_y = construct_y(gbp_data)

  
	X = construct_X(cny_data)
	print (X)
	
	

main()

[[-1.72617874e-02 -1.84960264e-03 -1.84960264e-03 ...  2.09314719e+00
   4.18612175e+00  6.27918263e+00]
 [ 2.46615206e-03 -4.11017742e-04 -4.11017742e-04 ...  2.09312664e+00
   4.18617108e+00  6.27925661e+00]
 [ 3.57523721e-02  8.62982971e-03  8.62982971e-03 ...  2.09320574e+00
   4.18688612e+00  6.28032918e+00]
 ...
 [ 4.54765069e-01  9.62986223e-02  8.01298229e-02 ...  1.91637505e+00
   3.82384970e+00  5.73577455e+00]
 [ 1.50638025e-01  1.22862330e-01  7.46548475e-02 ...  1.91663354e+00
   3.82686246e+00  5.74029369e+00]
 [ 9.58765773e-02  1.43522525e-01  9.01914338e-02 ...  1.91687019e+00
   3.82877999e+00  5.74316999e+00]]


In [17]:

# construct X_all
#[x_cny, x_jpy, x_eur, x_gbp]

def construct_X_all():
	
	cny_data = loadExcelData('data.xlsx','USDCNY')
	jpy_data = loadExcelData('data.xlsx','USDJPY')
	eur_data = loadExcelData('data.xlsx','USDEUR')
	gbp_data = loadExcelData('data.xlsx','USDGBP')

	cny_data = fillEmptyDates(cny_data)
	jpy_data = fillEmptyDates(jpy_data)
	eur_data = fillEmptyDates(eur_data)
	gbp_data = fillEmptyDates(gbp_data)

	X_cny = construct_X(cny_data)[-3500:]
	X_jpy = construct_X(jpy_data)[-3500:]
	X_eur = construct_X(eur_data)[-3500:]
	X_gbp = construct_X(gbp_data)[-3500:]

	X_all = np.hstack((X_cny, X_jpy, X_eur, X_gbp))

	return X_all


def construct_y_all(data):
	r = construct_return(data)
	y = np.sign(r[3:])
	zero_index = np.where(y == 0)[0]
	y[zero_index] = 1
	return y[-3500:]


	
# construct X
# [x1 x2 x3 x4 x5 x6 x7]
# [r SMA5 SMA10 SMA20 SMA60 SMA120 lDay]  
def construct_X(data):
	x1 = construct_return(data).reshape((-1,1))
	x2 = construct_SMA(x1,5).reshape((-1,1))
	x3 = construct_SMA(x1,10).reshape((-1,1))
	x4 = construct_SMA(x1,20).reshape((-1,1))
	x5 = construct_SMA(x1,60).reshape((-1,1))
	x6 = construct_SMA(x1,120).reshape((-1,1))
	x7 = construct_lastDay(x1).reshape((-1,1))
	x8 = np.log(data[:,1].astype(float)).reshape((-1,1))
	x9 = construct_SMA(x8,5).reshape((-1,1))
	x10 = construct_SMA(x8, 10).reshape((-1, 1))
	x11 = construct_SMA(x8, 20).reshape((-1, 1))
	x12 = construct_SMA(x8, 60).reshape((-1, 1))
	x13 = construct_SMA(x8, 220).reshape((-1, 1))
	x14= np.log(np.square(data[:,1].astype(float)).reshape((-1,1)))
	x15 = np.log(data[:, 1].astype(float).reshape((-1, 1))**3)
	X = np.hstack((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15))
	X = X[2:-1,:]

	return (X)


# return last day's closing price
def construct_lastDay(r):
	lDay = np.zeros_like(r)
	lDay[0:2] = None
	lDay[2:] = r[1:-1]
	return lDay


# for example: SMA5
# in the form of (r2, (r2+r3)/2, (r2+r3+r4)/3, (r2+r3+r4+r5)/4, (r2+r3+r4+r5+r6)/5, 
# (r3+r4+r5+r6+r7)/5, ...)
def construct_SMA(r,degree):
	num_days = r.shape[0]
	sma = np.zeros_like(r)
	for i in range(num_days):
		if i == 0:
			sma[i] = None
		elif i < degree:
			sma[i] = np.mean(r[1:i+1])
		else:
			sma[i] = np.mean(r[i-degree+1:i+1])
	return sma



# construct rs
# r0 = None
# start from r1 = 100*log(p1-p0)
def construct_return(data):
	p = data[:,1].astype(float)
	r = np.zeros_like(p)
	r[0] = None
	r[1:] = 100.0 * (np.log(p[1:])-np.log(p[:-1]))
	return r


# construct y
# y0 = None
# y1 = sign(r1)
def construct_y(data):
	r = construct_return(data)
	y = np.sign(r[3:])
	zero_index = np.where(y == 0)[0]
	y[zero_index] = 1
	return y


# Reads an Excel file 
# return: data = ['date', 'last price']
def loadExcelData(fileName, sheetName):
	data = pd.DataFrame(pd.read_excel(fileName, sheetName), columns= ['Date', 'Last Price'])
	return data.values



# Fill data in the empty dates (weekends) with the average
# of porior and posterior data
def fillEmptyDates(data):
	num_days = data.shape[0]
	empty_days_index = np.where(data[:,1] == 0)[0]
	empty_days_list = empty_days_index.tolist()
	filled_days_index = data[:,1].nonzero()[0]
	filled_days_list = filled_days_index.tolist
	for i in empty_days_list:
		if i == 0:
			data[i,3] = data[filled_days_index[0],1]

		elif i == num_days-1:
			data[i,3] = data[filled_days_index[-1],1]
		else:
			left_index = np.where(filled_days_index <= i)[0][-1]
			right_index = np.where(filled_days_index >= i)[0][0]
			data[i,1] = (data[filled_days_index[left_index],1] + data[filled_days_index[right_index],1])/2.0
	return data

# This if statement passes if this
# was the file that was executed
if __name__ == '__main__':
	main()


[[-1.72617874e-02 -1.84960264e-03 -1.84960264e-03 ...  2.09314719e+00
   4.18612175e+00  6.27918263e+00]
 [ 2.46615206e-03 -4.11017742e-04 -4.11017742e-04 ...  2.09312664e+00
   4.18617108e+00  6.27925661e+00]
 [ 3.57523721e-02  8.62982971e-03  8.62982971e-03 ...  2.09320574e+00
   4.18688612e+00  6.28032918e+00]
 ...
 [ 4.54765069e-01  9.62986223e-02  8.01298229e-02 ...  1.91637505e+00
   3.82384970e+00  5.73577455e+00]
 [ 1.50638025e-01  1.22862330e-01  7.46548475e-02 ...  1.91663354e+00
   3.82686246e+00  5.74029369e+00]
 [ 9.58765773e-02  1.43522525e-01  9.01914338e-02 ...  1.91687019e+00
   3.82877999e+00  5.74316999e+00]]


In [0]:

# The main method
def main():
	# load data = ['date', 'last price']
	cny_data = loadExcelData('data.xlsx','USDCNY')
	jpy_data = loadExcelData('data.xlsx','USDJPY')
	eur_data = loadExcelData('data.xlsx','USDEUR')
	gbp_data = loadExcelData('data.xlsx','USDGBP')
	
	# Fill data in the empty dates (weekends) with the average
	# of porior and posterior data
	cny_data = fillEmptyDates(cny_data)
	jpy_data = fillEmptyDates(jpy_data)
	eur_data = fillEmptyDates(eur_data)
	gbp_data = fillEmptyDates(gbp_data)

	# construct retrun values (r2,...,rn)
	cny_r = construct_return(cny_data)
	jpy_r = construct_return(jpy_data)
	eur_r = construct_return(eur_data)
	gbp_r = construct_return(gbp_data)

	# construct sma
	cny_sma5 = construct_SMA(cny_r,5)
	jpy_sma5 = construct_SMA(jpy_r,5)
	eur_sma5 = construct_SMA(eur_r,5)
	gbp_sma5 = construct_SMA(gbp_r,5)

	# construct last-day-closing-price structure
	cny_lDay = construct_lastDay(cny_r)
	jpy_lDay = construct_lastDay(jpy_r)
	eur_lDay = construct_lastDay(eur_r)
	gbp_lDay = construct_lastDay(gbp_r)

	# construct y 
	cny_y = construct_y(cny_data)
	jpy_y = construct_y(jpy_data)	
	eur_y = construct_y(eur_data)
	gbp_y = construct_y(gbp_data)


In [0]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model





def split_data(currency):
  data = loadExcelFile('data.xlsx', currency)
  x = construct_X(data)
  r = construct_return(data)[3:]
  
  
  
  


In [0]:

def split_data(currency):
	# Load data for currency
	data = dp.loadExcelData('data.xlsx', currency)
	X = dp.construct_X(data)
	r = dp.construct_return(data)[3:]
	
	# number of datapoints # resume point1
	n = r.shape[0]
	_n = n//10
	
	# Split the data into training/validate/testing sets
	X_train = X[:8*_n]
	X_val = X[8*_n:9*_n]
	X_test = X[9*_n:]

	# Split the targets into training/validate/testing sets
	r_train = r[:8*_n]
	r_val = r[8*_n:9*_n]
	r_test = r[9*_n:]
	
	return X_train, X_val, X_test, r_train, r_val, r_test


if __name__ == '__main__':
	main()


In [0]:
# number of datapoints
	#n = r.shape[0]
	_#n = n//10
	